In [1]:
import numpy as np # linear algebra
import scipy as scipy
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import dill

import keras as k
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

import cv2
from tqdm import tqdm


Using Theano backend.


In [2]:
with open('tiffs.pkl', 'rb') as in_strm:
    all_info = dill.load(in_strm)
x_train = all_info[0]
x_test = all_info[1]
y_train = all_info[2]
y_test = all_info[3]

print(x_train.shape)

(35000, 32, 32, 4)


In [4]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                input_shape=(32, 32, 4)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))

model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
                    optimizer='adam',
                    metrics=['accuracy'])


model.fit(x_train, y_train, batch_size=128, epochs=7, verbose=1, 
          validation_data=(x_test, y_test))

from sklearn.metrics import fbeta_score

p_valid = model.predict(x_test, batch_size=128)
print(y_test)
print(p_valid)
print(fbeta_score(y_test, np.array(p_valid) > 0.2, beta=2, average='samples'))

Train on 35000 samples, validate on 5479 samples
Epoch 1/7
35000/35000 [==============================] - 205s - loss: 0.3446 - acc: 0.8865 - val_loss: 0.2412 - val_acc: 0.9078
Epoch 2/7
35000/35000 [==============================] - 203s - loss: 0.2457 - acc: 0.9051 - val_loss: 0.2211 - val_acc: 0.9082
Epoch 3/7
35000/35000 [==============================] - 251s - loss: 0.2339 - acc: 0.9069 - val_loss: 0.2189 - val_acc: 0.9081
Epoch 4/7
35000/35000 [==============================] - 234s - loss: 0.2264 - acc: 0.9081 - val_loss: 0.2174 - val_acc: 0.9092
Epoch 5/7
35000/35000 [==============================] - 221s - loss: 0.2216 - acc: 0.9089 - val_loss: 0.2132 - val_acc: 0.9119
Epoch 6/7
35000/35000 [==============================] - 220s - loss: 0.2195 - acc: 0.9096 - val_loss: 0.2091 - val_acc: 0.9116
Epoch 7/7
35000/35000 [==============================] - 249s - loss: 0.2165 - acc: 0.9106 - val_loss: 0.2047 - val_acc: 0.9143
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 1 0]
 [0 0 1 ..., 0 

In [1]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')



for i in inv_label_map.keys():
    label = inv_label_map[i]
    cnf_matrix = confusion_matrix(y_test[:,i], p_valid[:,i]>0.2)
    plot_confusion_matrix(cnf_matrix, classes=[label], normalize=False,
                      title=label)
    total = cnf_matrix[0][0] + cnf_matrix[0][1] + cnf_matrix[1][0] + cnf_matrix[1][1]
    actualYes = cnf_matrix[1][0] + cnf_matrix[1][1]
    actualNo = cnf_matrix[0][0] + cnf_matrix[0][1]
    predYes = cnf_matrix[0][1] + cnf_matrix[1][1]
    plt.show()
    print("Stats for ", label)
    print("Accuracy (true positive + true negative) / total: ", (cnf_matrix[1][1] + cnf_matrix[0][0])/total)
    print("FPR (when actually no, how often does it say yes): ", cnf_matrix[0][1]/actualNo)
    print("TPR aka Recall (when actually yes, how often does it say yes): ", cnf_matrix[1][1]/actualYes)
    print("Precision (when predicts yes, how often is it right): ", cnf_matrix[1][1]/predYes)
    print("Prevalence (how often does actual yes occur in sample): ", actualYes/total)

NameError: name 'inv_label_map' is not defined